In [1]:
from book_model import BookAllField

## 2.5.10. Atomic updates(原子更新)

Documents may be updated atomically by using the `update_one()`, `update()` and `modify()` methods on a `QuerySet` or `modify()` and `save()` (with save_condition argument) on a Document. There are several different “modifiers” that you may use with these methods:

`set` – set a particular value：设置特定值  
`set_on_insert` – set only if this is new document `need to add upsert=True`_：只有在新文档时更新值  
`unset` – delete a particular value (since MongoDB v1.3)：删除特定值  
`max` – update only if value is bigger：较大时更新  
`min` – update only if value is smaller：较小时更新  
`inc` – increment a value by a given amount：将一个值增加给定数量  
`dec` – decrement a value by a given amount：将一个值减少给定数量  
`push` – append a value to a list：添加值到列表末尾  
`push_all` – append several values to a list：添加多个值到列表末尾  
`pop` – remove the first or last element of a list depending on the value：删除列表开头或末尾的一个值  
`pull` – remove a value from a list：从列表中删除给定的值  
`pull_all` – remove several values from a list：从列表中删除给定的多个值    
`add_to_set` – add value to a list only if its not in the list already：仅在列表中没有这个值时添加  
`rename` – rename the key name：修改键名

The syntax for atomic updates is similar to the querying syntax, but the modifier comes before the field, not after it:

原子更新的操作符与查询符号很像，但更新操作符在字段之前(与查询相同，使用双下划线连接字段名称)

```python
post = BlogPost(title='Test', page_views=0, tags=['database'])
post.save()
BlogPost.objects(id=post.id).update_one(inc__page_views=1)
post.reload()  # the document has been changed, so we need to reload it
post.page_views
1
BlogPost.objects(id=post.id).update_one(set__title='Example Post')
post.reload()
post.title
'Example Post'
BlogPost.objects(id=post.id).update_one(push__tags='nosql')
post.reload()
post.tags
['database', 'nosql']
```

### modifier operator: Inc and Dec

In [18]:
BookAllField.objects(name="大学", publishing_house__name="人民教育出版社")[0].price

10.8

In [19]:
# inc 操作符，在原有的价格上加 1;在 QuerySet 上执行 update_one,而不是 Document 上
BookAllField.objects(name="大学", publishing_house__name="人民教育出版社").update_one(inc__price=1)

1

In [20]:
# 查询更新后的价格
BookAllField.objects(name="大学", publishing_house__name="人民教育出版社")[0].price

11.8

In [21]:
# inc 操作符，在原有的价格上减 1
BookAllField.objects(name="大学", publishing_house__name="人民教育出版社").update_one(dec__price=1)

1

In [22]:
# 查询更新后的价格
BookAllField.objects(name="大学", publishing_house__name="人民教育出版社")[0].price

10.8

### modifier operator:push, push_all; pull, pull_all

In [23]:
BookAllField.objects(name="大学", publishing_house__name="人民教育出版社")[0].tag

['历史', '经典', '必读']

In [27]:
# 列表中添加最后一个值
BookAllField.objects(name="大学", publishing_house__name="人民教育出版社").update_one(push__tag="儒学")

1

In [28]:
BookAllField.objects(name="大学", publishing_house__name="人民教育出版社")[0].tag

['历史', '经典', '必读', '儒学']

In [29]:
# 删除 list 中最后一个值
BookAllField.objects(name="大学", publishing_house__name="人民教育出版社").update_one(pull__tag="儒学")

1

In [30]:
BookAllField.objects(name="大学", publishing_house__name="人民教育出版社")[0].tag

['历史', '经典', '必读']

In [31]:
# 列表后，添加多个元素
BookAllField.objects(name="大学", publishing_house__name="人民教育出版社").update_one(push_all__tag=["儒学","四书"])

1

In [32]:
BookAllField.objects(name="大学", publishing_house__name="人民教育出版社")[0].tag

['历史', '经典', '必读', '儒学', '四书']

In [33]:
# 列表后，删除多个元素
BookAllField.objects(name="大学", publishing_house__name="人民教育出版社").update_one(pull_all__tag=["儒学","四书"])

1

In [34]:
BookAllField.objects(name="大学", publishing_house__name="人民教育出版社")[0].tag

['历史', '经典', '必读']

### modifier operator:set

In [35]:
BookAllField.objects(name="大学", publishing_house__name="人民教育出版社").to_json(ensure_ascii=False)

'[{"_id": {"$oid": "6408686b248d96a15f6ff2e4"}, "bookid": "wx0927615ssr", "name": "大学", "ifnewbook": false, "shelves_date": {"$date": 1608422400000}, "author": {"$oid": "6408686b248d96a15f6ff2e2"}, "author_email": "zengzi@qq.com", "price": 10.8, "book_type": {"$ref": "book_type", "$id": {"$oid": "63fcd4f21be34fe2c671fb44"}}, "publishing_house": {"name": "人民教育出版社", "phone": "120934876"}, "seller": {"$ref": "seller", "$id": {"$oid": "6408686b248d96a15f6ff2e3"}}, "tag": ["历史", "经典", "必读"]}]'

In [36]:
# 使用set 设置值
BookAllField.objects(name="大学", publishing_house__name="人民教育出版社").update_one(set__author_email="abc@126.com")

1

In [37]:
BookAllField.objects(name="大学", publishing_house__name="人民教育出版社").to_json(ensure_ascii=False)

'[{"_id": {"$oid": "6408686b248d96a15f6ff2e4"}, "bookid": "wx0927615ssr", "name": "大学", "ifnewbook": false, "shelves_date": {"$date": 1608422400000}, "author": {"$oid": "6408686b248d96a15f6ff2e2"}, "author_email": "abc@126.com", "price": 10.8, "book_type": {"$ref": "book_type", "$id": {"$oid": "63fcd4f21be34fe2c671fb44"}}, "publishing_house": {"name": "人民教育出版社", "phone": "120934876"}, "seller": {"$ref": "seller", "$id": {"$oid": "6408686b248d96a15f6ff2e3"}}, "tag": ["历史", "经典", "必读"]}]'

### Default modifier operator  
If no modifier operator is specified the default will be `$set`. So the following sentences are identical:

```python
BlogPost.objects(id=post.id).update(title='Example Post')
BlogPost.objects(id=post.id).update(set__title='Example Post')
```

### push list position

From MongoDB version 2.6, push operator supports `$position` value which allows to push values with index::

```python
post = BlogPost(title="Test", tags=["mongo"])
post.save()
post.update(push__tags__0=["database", "code"])
post.reload()
post.tags
['database', 'code', 'mongo']
```